<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Лемматизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разделение на выборки</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Градиентный-бустинг-Catboost" data-toc-modified-id="Градиентный-бустинг-Catboost-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Градиентный бустинг Catboost</a></span></li><li><span><a href="#Работа-с-дисбалансом" data-toc-modified-id="Работа-с-дисбалансом-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Работа с дисбалансом</a></span></li><li><span><a href="#Вывод-по-результатам-обучения" data-toc-modified-id="Вывод-по-результатам-обучения-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Вывод по результатам обучения</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

## Подготовка

### Загрузка данных

In [2]:
import re
import time

import pandas as pd

from catboost import CatBoostClassifier

from imblearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from imblearn.over_sampling import RandomOverSampler,  SMOTE

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  f1_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline

import nltk
from nltk.corpus import stopwords as nltk_stopwords

import spacy

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [3]:
try:
     df = pd.read_csv(r"C:\Проекты Яндекс.Практикум\13. Машинное обучение для текстов\Датасет\toxic_comments.csv")
except: 
    df=pd.read_csv('/datasets/toxic_comments.csv')

Загрузили необходимые библиотеки и датасет

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [7]:
df['Unnamed: 0'].describe()

count    159292.000000
mean      79725.697242
std       46028.837471
min           0.000000
25%       39872.750000
50%       79721.500000
75%      119573.250000
max      159450.000000
Name: Unnamed: 0, dtype: float64

Датасет состоит из трех столбцов:
- text, с комментариями на английском
- toxic, с классификацией по токсичности. Положительного класса 16 тыс записей, отрицательного 143 тыс. Разница в 8.8 раз
- Unnamed: 0, неинформативный столбеЦ, который можно удалить. Данные в столбце частично дублируют индексы

In [8]:
df = df.drop(['Unnamed: 0'], axis=1)

Удалили столбец Unnamed: 0

### Лемматизация

In [9]:
new_corpus = []
nlp = spacy.load('en_core_web_sm')

for doc in tqdm(nlp.pipe(df['text'], batch_size=64, n_process=-1, disable=["parser", "ner"]), total=len(df['text'])):
    word_list = [tok.lemma_ for tok in doc]
    new_corpus.append(' '.join(word_list))
    
df['lemm_spacy_new'] = new_corpus 

100%|█████████████████████████████████████████████████████████████████████████| 159292/159292 [14:38<00:00, 181.32it/s]


In [10]:
tqdm.pandas()

def clear_text(text):
  cleared_text = re.sub(r'[^a-zA-Z]', ' ',text)
  cleared_text = cleared_text.lower()
  return ' '.join(cleared_text.split())

df['clear_text'] = df['lemm_spacy_new'].progress_apply(clear_text)

100%|███████████████████████████████████████████████████████████████████████| 159292/159292 [00:09<00:00, 17138.53it/s]


In [11]:
df.head()

,text,toxic,lemm_spacy_new,clear_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation \n why the edit make under my user...,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! he match this background colour I be s...,d aww he match this background colour i be see...
2,"Hey man, I'm really not trying to edit war. It...",0,"hey man , I be really not try to edit war . it...",hey man i be really not try to edit war it be ...
3,"""\nMore\nI can't make any real suggestions on ...",0,""" \n More \n I can not make any real suggestio...",more i can not make any real suggestion on imp...
4,"You, sir, are my hero. Any chance you remember...",0,"you , sir , be my hero . any chance you rememb...",you sir be my hero any chance you remember wha...


In [12]:
df.head()

,text,toxic,lemm_spacy_new,clear_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation \n why the edit make under my user...,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! he match this background colour I be s...,d aww he match this background colour i be see...
2,"Hey man, I'm really not trying to edit war. It...",0,"hey man , I be really not try to edit war . it...",hey man i be really not try to edit war it be ...
3,"""\nMore\nI can't make any real suggestions on ...",0,""" \n More \n I can not make any real suggestio...",more i can not make any real suggestion on imp...
4,"You, sir, are my hero. Any chance you remember...",0,"you , sir , be my hero . any chance you rememb...",you sir be my hero any chance you remember wha...


## Обучение

### Разделение на выборки

In [13]:
features = df['clear_text']
target = df['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.3, random_state=42) 

### Логистическая регрессия

In [14]:
%%time
start = time.time()

stopwords = set(nltk_stopwords.words('english'))

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('logreg', LogisticRegression(random_state=42)),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'logreg__C': [1,2, 6], 
    'logreg__class_weight': [None, 'balanced']
    
}

grid_search_tune = RandomizedSearchCV(pipeline, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(features_train, target_train)
print(grid_search_tune.best_score_, 'лучшие параметры:', grid_search_tune.best_params_)


end = time.time()
round(end-start,2)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
0.7696410556545855 лучшие параметры: {'tfidf__ngram_range': (1, 3), 'tfidf__max_df': 0.5, 'logreg__class_weight': 'balanced', 'logreg__C': 6}
Wall time: 38min 34s


2314.76

### Случайный лес

In [15]:
%%time
start = time.time()

stopwords = set(nltk_stopwords.words('english'))

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('ranfor', RandomForestClassifier(random_state=42)),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75, 0.9),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'ranfor__max_depth': [3,13,2], 
    'ranfor__n_estimators': [10,200,10], 
    'ranfor__class_weight': [None, 'balanced']
    
}

grid_search_tune = RandomizedSearchCV(pipeline, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(features_train, target_train)
print(grid_search_tune.best_score_, 'лучшие параметры:', grid_search_tune.best_params_)


end = time.time()
round(end-start,2)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
0.3284908223884859 лучшие параметры: {'tfidf__ngram_range': (1, 1), 'tfidf__max_df': 0.25, 'ranfor__n_estimators': 200, 'ranfor__max_depth': 3, 'ranfor__class_weight': 'balanced'}
Wall time: 8min 36s


516.45

### Градиентный бустинг Catboost

Ссылка на CatBoostClassifier, проведенный в Google Colab https://colab.research.google.com/drive/1__Qdi36DVzOjeQrn7miZYsrOask5zRPZ?usp=sharing

### Работа с дисбалансом

In [16]:
pipeline_oversampling = imbPipeline([
    ('tfidf', TfidfVectorizer()),
    ('oversampling', RandomOverSampler(random_state=0)),
    ('logreg', LogisticRegression(random_state=42))])


parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'logreg__C': [1,2,6]
}

grid_search_tune = RandomizedSearchCV(pipeline_oversampling, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(features_train, target_train)

print(f"Best score: {abs(grid_search_tune.best_score_)}, Best params: {grid_search_tune.best_params_}")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best score: 0.775580166595057, Best params: {'tfidf__ngram_range': (1, 2), 'tfidf__max_df': 0.25, 'logreg__C': 2}


### Вывод по результатам обучения

Разделили данные на обучаюшую (0.7) и тестовую (0.3) выборки. 
На каждой модели провели векторизацию и обучение с подбором гиперпараметров (в том числе с балансировкой классов) через  RandomizedSearchCV.
- Логистическая регрессия: F1 = 0.769, гиперпараметры векторизатора: ngram_range = (1, 3), max_df = 0.5, гиперпараметы модели class_weight = balanced,  C = 6.
- Случайный лес: F1 = 0.328, гиперпараметры векторизатора: ngram_range = (1, 1), max_df = 0.25, гиперпараметы модели  n_estimators = 200, max_depth = 3,  class_weight = balanced 
- CatBoost : F1 = 0.774, гиперпараметры модели: class_weights =[1, 8.87], depth = 10, l2_leaf_reg = 3, learning_rate = 0.3
- Логистическая регрессия с уменьшением отрицательного класса: F1 = Best score: 0.775 гиперпараметры векторизатора: ngram_range = (1, 2), max_df = 0.25, гиперпараметы модели: C = 2.

Для тестирования выбрана первая модель (Логистическая регрессия), как более стабильная по сравнению с Логистической регрессией обученной на уменьшенной выборке. И более быстрая и легкая  в обучении по сравнению с CatBoost

### Тестирование

In [19]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords , max_df = 0.5, ngram_range = (1, 3))
tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test) 

In [20]:
model = LogisticRegression(random_state=42, C = 6, class_weight = 'balanced')
model.fit(tf_idf_train , target_train)
predict_log_test = model.predict(tf_idf_test )
f_1_lof_test=f1_score(predict_log_test, target_test )
print('Логистическая регрессия - F1 на тестовой выборке:', f_1_lof_test)

Логистическая регрессия - F1 на тестовой выборке: 0.7695789059425423


## Общий вывод

Проект выполнен для интернет-магазина «Викишоп». Магазин запустил сервис редактирования описания товаров клиентами.
В процессе выполения проекта решалась задача обучения модели классифицировать описание на позитивные и негативные. Язык комментариев - английский.

Был получен датасет со 159292 комментариями с классификацией токсичности. Данные были очищены от неифоративного столбца. Тексты комментариев были лемматизированы с помощью лемматизатора Spacy.

Обучили несколько моделей. На каждой модели провели векторизацию и обучение с подбором гиперпараметров (в том числе с балансировкой классов) через RandomizedSearchCV.

- Логистическая регрессия: F1 = 0.769, гиперпараметры векторизатора: ngram_range = (1, 3), max_df = 0.5, гиперпараметы модели class_weight = balanced, C = 6.
- Случайный лес: F1 = 0.328, гиперпараметры векторизатора: ngram_range = (1, 1), max_df = 0.25, гиперпараметы модели n_estimators = 200, max_depth = 3, class_weight = balanced
- CatBoost : F1 = 0.774, гиперпараметры модели: class_weights =[1, 8.87], depth = 10, l2_leaf_reg = 3, learning_rate = 0.3
- Логистическая регрессия с уменьшением отрицательного класса: F1 = Best score: 0.775 гиперпараметры векторизатора: ngram_range = (1, 2), max_df = 0.25, гиперпараметы модели: C = 2.

Для тестирования выбрана первая модель (Логистическая регрессия), как более стабильная по сравнению с Логистической регрессией обученной на уменьшенной выборке. И более быстрая и легкая в обучении по сравнению с CatBoost.

Проверенная не тестировании модель дала результат F1 = 0.769. Модель пригодна для использования.